In [20]:
import requests
import pandas as pd
import pymongo

In [21]:
url = "https://api.spacexdata.com/v4/launches/upcoming"


In [22]:
results = requests.get(url).json()

In [23]:
space=[]
for result in results:
    #print(results["flight_number"])
    space.append({"flight_id": result["id"],
                "launch_date": result["date_local"],
                "launch_pad": result["launchpad"]})

In [24]:
space_df=pd.DataFrame(space)

In [25]:
space_df

,flight_id,launch_date,launch_pad
0,60428aafc041c16716f73cd7,2021-03-24T04:58:00-04:00,5e9e4501f509094ba4566f84
1,60428ac4c041c16716f73cd8,2021-02-28T19:00:00-05:00,5e9e4501f509094ba4566f84
2,5fe3af58b3467846b324215f,2021-04-22T06:11:00-04:00,5e9e4502f509094188566f88
3,5fe3af43b3467846b324215e,2021-04-30T17:00:00-07:00,5e9e4502f509092b78566f87
4,600f9b6d8f798e2a4d5f979f,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84
5,5fe3af84b3467846b3242161,2021-05-31T20:00:00-04:00,5e9e4502f509094188566f88
6,5fe3afc1b3467846b3242164,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84
7,5fe3af6db3467846b3242160,2021-03-31T20:00:00-04:00,5e9e4501f509094ba4566f84
8,5fe3af99b3467846b3242162,2021-06-30T20:00:00-04:00,5e9e4502f509094188566f88
9,5eb87d4effd86e000604b390,2021-06-30T20:00:00-04:00,5e9e4501f509094ba4566f84


In [26]:
url_b="https://api.spacexdata.com/v4/launchpads"
results_b = requests.get(url_b).json()
results_b

[{'name': 'VAFB SLC 3W',
  'full_name': 'Vandenberg Air Force Base Space Launch Complex 3W',
  'locality': 'Vandenberg Air Force Base',
  'region': 'California',
  'timezone': 'America/Los_Angeles',
  'latitude': 34.6440904,
  'longitude': -120.5931438,
  'launch_attempts': 0,
  'launch_successes': 0,
  'rockets': ['5e9d0d95eda69955f709d1eb'],
  'launches': [],
  'details': "SpaceX's original west coast launch pad for Falcon 1. It was used in a static fire test but was never employed for a launch, and was abandoned due to range scheduling conflicts arising from overflying other active pads.",
  'status': 'retired',
  'id': '5e9e4501f5090910d4566f83'},
 {'name': 'CCSFS SLC 40',
  'full_name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
  'locality': 'Cape Canaveral',
  'region': 'Florida',
  'timezone': 'America/New_York',
  'latitude': 28.5618571,
  'longitude': -80.577366,
  'launch_attempts': 67,
  'launch_successes': 65,
  'rockets': ['5e9d0d95eda69973a809d1ec'],
 

In [27]:
pads = []
#results
for result in results_b:
    pads.append({"Name":result["full_name"],
                 "launch_pad": result["id"],
                 "lat": result["latitude"],
                 "long": result["longitude"]})

In [28]:
pads

[{'Name': 'Vandenberg Air Force Base Space Launch Complex 3W',
  'launch_pad': '5e9e4501f5090910d4566f83',
  'lat': 34.6440904,
  'long': -120.5931438},
 {'Name': 'Cape Canaveral Space Force Station Space Launch Complex 40',
  'launch_pad': '5e9e4501f509094ba4566f84',
  'lat': 28.5618571,
  'long': -80.577366},
 {'Name': 'SpaceX South Texas Launch Site',
  'launch_pad': '5e9e4502f5090927f8566f85',
  'lat': 25.9972641,
  'long': -97.1560845},
 {'Name': 'Kwajalein Atoll Omelek Island',
  'launch_pad': '5e9e4502f5090995de566f86',
  'lat': 9.0477206,
  'long': 167.7431292},
 {'Name': 'Vandenberg Air Force Base Space Launch Complex 4E',
  'launch_pad': '5e9e4502f509092b78566f87',
  'lat': 34.632093,
  'long': -120.610829},
 {'Name': 'Kennedy Space Center Historic Launch Complex 39A',
  'launch_pad': '5e9e4502f509094188566f88',
  'lat': 28.6080585,
  'long': -80.6039558}]

In [29]:
pads_df=pd.DataFrame(pads)

In [30]:
launch_location=space_df.merge(pads_df,on="launch_pad")

In [31]:
launch_location

,flight_id,launch_date,launch_pad,Name,lat,long
0,60428aafc041c16716f73cd7,2021-03-24T04:58:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
1,60428ac4c041c16716f73cd8,2021-02-28T19:00:00-05:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
2,600f9b6d8f798e2a4d5f979f,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
3,5fe3afc1b3467846b3242164,2021-05-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
4,5fe3af6db3467846b3242160,2021-03-31T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
5,5eb87d4effd86e000604b390,2021-06-30T20:00:00-04:00,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.561857,-80.577366
6,5fe3af58b3467846b324215f,2021-04-22T06:11:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
7,5fe3af84b3467846b3242161,2021-05-31T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
8,5fe3af99b3467846b3242162,2021-06-30T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956
9,5fe3b11eb3467846b324216c,2021-08-17T20:00:00-04:00,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.608058,-80.603956


In [32]:
launch_location[['launch_date', 'launch_time']] = launch_location['launch_date'].str.split('T', expand=True)
launch_location=launch_location.round({'lat':2, 'long':2})
launch_location

,flight_id,launch_date,launch_pad,Name,lat,long,launch_time
0,60428aafc041c16716f73cd7,2021-03-24,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,04:58:00-04:00
1,60428ac4c041c16716f73cd8,2021-02-28,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,19:00:00-05:00
2,600f9b6d8f798e2a4d5f979f,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,20:00:00-04:00
3,5fe3afc1b3467846b3242164,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,20:00:00-04:00
4,5fe3af6db3467846b3242160,2021-03-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,20:00:00-04:00
5,5eb87d4effd86e000604b390,2021-06-30,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,28.56,-80.58,20:00:00-04:00
6,5fe3af58b3467846b324215f,2021-04-22,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.61,-80.60,06:11:00-04:00
7,5fe3af84b3467846b3242161,2021-05-31,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.61,-80.60,20:00:00-04:00
8,5fe3af99b3467846b3242162,2021-06-30,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.61,-80.60,20:00:00-04:00
9,5fe3b11eb3467846b324216c,2021-08-17,5e9e4502f509094188566f88,Kennedy Space Center Historic Launch Complex 39A,28.61,-80.60,20:00:00-04:00


In [33]:
weath_url="http://api.weatherbit.io/v2.0/forecast/daily?key=24bb5c9cb97c4a06a04d2a1bba2b75aa&lat=28.561857&lon=-80.577366"
weath_results = requests.get(weath_url).json()
weath_results

{'data': [{'moonrise_ts': 1616255341,
   'wind_cdir': 'W',
   'rh': 89,
   'pres': 1018.2083,
   'high_temp': 19.1,
   'sunset_ts': 1616283261,
   'ozone': 292.7262,
   'moon_phase': 0.466474,
   'wind_gust_spd': 18.796875,
   'snow_depth': 0,
   'clouds': 67,
   'ts': 1616212860,
   'sunrise_ts': 1616239519,
   'app_min_temp': 16.2,
   'wind_spd': 10.221977,
   'pop': 85,
   'wind_cdir_full': 'west',
   'slp': 1018.2083,
   'moon_phase_lunation': 0.26,
   'valid_date': '2021-03-20',
   'app_max_temp': 19.3,
   'vis': 20.650333,
   'dewpt': 15.6,
   'snow': 0,
   'uv': 4.752395,
   'weather': {'icon': 'r01d', 'code': 500, 'description': 'Light rain'},
   'wind_dir': 274,
   'max_dhi': None,
   'clouds_hi': 0,
   'precip': 6.375,
   'low_temp': 12.5,
   'max_temp': 19.2,
   'moonset_ts': 1616220584,
   'datetime': '2021-03-20',
   'temp': 17.5,
   'min_temp': 16.2,
   'clouds_mid': 0,
   'clouds_low': 67},
  {'moonrise_ts': 1616344527,
   'wind_cdir': 'W',
   'rh': 83,
   'pres': 1016.5

In [34]:
weather = []
params = {
    "key": "24bb5c9cb97c4a06a04d2a1bba2b75aa"
}

weath_url="http://api.weatherbit.io/v2.0/forecast/daily"

for index, row in launch_location.iterrows():
    params["lat"]= row["lat"]
    params["lon"]= row["long"]
    address= requests.get(weath_url, params=params)
    address_j=address.json()
    
    for result in address_j["data"]:
        post= {"lat": address_j["lat"],
               "long":address_j["lon"],
               "Valid_date":result["valid_date"],
               "Ave_Temp": result["temp"],
               "Max_Temp":result["max_temp"],
               "Min_Temp": result["min_temp"],
               "H_Level_Cloud":result["clouds_hi"],
               "Wind_Gust_Spd":result["wind_gust_spd"],
               "weather_condition":result["weather"]["description"]}
        weather.append(post)
weather



[{'lat': 28.56,
  'long': -80.58,
  'Valid_date': '2021-03-20',
  'Ave_Temp': 17.5,
  'Max_Temp': 19.2,
  'Min_Temp': 16.2,
  'H_Level_Cloud': 0,
  'Wind_Gust_Spd': 18.796875,
  'weather_condition': 'Light rain'},
 {'lat': 28.56,
  'long': -80.58,
  'Valid_date': '2021-03-21',
  'Ave_Temp': 15.7,
  'Max_Temp': 20.4,
  'Min_Temp': 12.4,
  'H_Level_Cloud': 0,
  'Wind_Gust_Spd': 13.8984375,
  'weather_condition': 'Scattered clouds'},
 {'lat': 28.56,
  'long': -80.58,
  'Valid_date': '2021-03-22',
  'Ave_Temp': 14.1,
  'Max_Temp': 16.8,
  'Min_Temp': 12.2,
  'H_Level_Cloud': 0,
  'Wind_Gust_Spd': 5.796875,
  'weather_condition': 'Scattered clouds'},
 {'lat': 28.56,
  'long': -80.58,
  'Valid_date': '2021-03-23',
  'Ave_Temp': 21.7,
  'Max_Temp': 25.8,
  'Min_Temp': 18.2,
  'H_Level_Cloud': 32,
  'Wind_Gust_Spd': 9.734375,
  'weather_condition': 'Scattered clouds'},
 {'lat': 28.56,
  'long': -80.58,
  'Valid_date': '2021-03-24',
  'Ave_Temp': 23,
  'Max_Temp': 27.7,
  'Min_Temp': 19.3,
  'H

In [35]:
weather_df=pd.DataFrame(weather)
weather_df

,lat,long,Valid_date,Ave_Temp,Max_Temp,Min_Temp,H_Level_Cloud,Wind_Gust_Spd,weather_condition
0,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain
1,28.56,-80.58,2021-03-21,15.7,20.4,12.4,0,13.898438,Scattered clouds
2,28.56,-80.58,2021-03-22,14.1,16.8,12.2,0,5.796875,Scattered clouds
3,28.56,-80.58,2021-03-23,21.7,25.8,18.2,32,9.734375,Scattered clouds
4,28.56,-80.58,2021-03-24,23.0,27.7,19.3,96,12.609375,Overcast clouds
...,...,...,...,...,...,...,...,...,...
219,34.63,-120.61,2021-03-31,12.9,16.6,6.0,12,11.015625,Few clouds
220,34.63,-120.61,2021-04-01,10.7,13.4,5.4,9,11.000000,Few clouds
221,34.63,-120.61,2021-04-02,9.7,11.2,5.4,0,11.710938,Few clouds
222,34.63,-120.61,2021-04-03,8.7,16.4,5.2,0,7.148438,Scattered clouds


In [36]:
launch_location_weather=weather_df.merge(launch_location, how="left", on=("lat", "long"))

In [37]:
launch_location_weather.drop_duplicates(inplace=True)
launch_location_weather = launch_location_weather.reset_index(drop=True)
launch_location_weather

,lat,long,Valid_date,Ave_Temp,Max_Temp,Min_Temp,H_Level_Cloud,Wind_Gust_Spd,weather_condition,flight_id,launch_date,launch_pad,Name,launch_time
0,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain,60428aafc041c16716f73cd7,2021-03-24,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,04:58:00-04:00
1,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain,60428ac4c041c16716f73cd8,2021-02-28,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,19:00:00-05:00
2,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain,600f9b6d8f798e2a4d5f979f,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00
3,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain,5fe3afc1b3467846b3242164,2021-05-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00
4,28.56,-80.58,2021-03-20,17.5,19.2,16.2,0,18.796875,Light rain,5fe3af6db3467846b3242160,2021-03-31,5e9e4501f509094ba4566f84,Cape Canaveral Space Force Station Space Launc...,20:00:00-04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
219,34.63,-120.61,2021-04-02,9.7,11.2,5.4,0,11.710938,Few clouds,5fe3b107b3467846b324216b,2021-11-23,5e9e4502f509092b78566f87,Vandenberg Air Force Base Space Launch Complex 4E,16:00:00-08:00
220,34.63,-120.61,2021-04-03,8.7,16.4,5.2,0,7.148438,Scattered clouds,5fe3af43b3467846b324215e,2021-04-30,5e9e4502f509092b78566f87,Vandenberg Air Force Base Space Launch Complex 4E,17:00:00-07:00
221,34.63,-120.61,2021-04-03,8.7,16.4,5.2,0,7.148438,Scattered clouds,5fe3b107b3467846b324216b,2021-11-23,5e9e4502f509092b78566f87,Vandenberg Air Force Base Space Launch Complex 4E,16:00:00-08:00
222,34.63,-120.61,2021-04-04,12.0,16.3,7.7,0,2.802734,Clear Sky,5fe3af43b3467846b324215e,2021-04-30,5e9e4502f509092b78566f87,Vandenberg Air Force Base Space Launch Complex 4E,17:00:00-07:00


In [48]:
 # The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Define the 'classDB' database in Mongo
db = client.spacex_launchDB

# Define Collection
launches = db.launches

In [52]:
for index, row in launch_location_weather.iterrows():
    launch_data={    "flight_id" : row['flight_id'],
        "launch_date" : row['launch_date'],
        "launch_date" : row['launch_date'],
        "launchpad_id" : row['launch_pad'],
        "launchpad_name" : row['Name'],
        "launch_time" : row['launch_time'],
        "weather_date" : row['Valid_date'],
        "lat" : row['lat'],
        "long" : row['long'],
        "weather_condition" : row['weather_condition'],
        "avg_temp_C" : row['Ave_Temp'],
        "max_temp_C" : row['Max_Temp'],
        "min_temp_C" : row['Min_Temp'],
        "high_level_clouds" : row['H_Level_Cloud'],
        "wind_gust_spd_MPS" : row['Wind_Gust_Spd'],
         }
        
#    print(launch_data)
    launches.insert_one(launch_data)
    